In [1]:
import os
import os.path as osp
import json
import h5py
import open3d as o3d
import math
import numpy as np
from tqdm import tqdm
import logging, sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

In [3]:
def get_object_nodes(sample_data):
    object_nodes = []
    for idx in range(len(sample_data['object_nodes'])):
        object_node = {}
        node_data = sample_data['object_nodes'][str(idx)]
        for key in node_data.keys():
            if node_data[key].shape is None:
                continue
            object_node[key] = node_data[key][:]
        object_nodes.append(object_node)
    return object_nodes

In [4]:
def readData(dataPath):
    meshPaths = []
    # giving file extension
    ext = ('.obj')
    for file_name in os.listdir(dataPath):
        if file_name.endswith(ext):
            meshPaths.append(osp.join(dataPath, file_name))
    return meshPaths

In [5]:
def getBBFromMeshPath(meshPath):
    mesh = o3d.io.read_triangle_mesh(meshPath)
    pcd = mesh.sample_points_uniformly(number_of_points=1024)
    bb = o3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd.points)
    return bb

In [6]:
def compare_BBs(shape_BB_size, object_BB_size, threshold):
    object_x_y_ratio = object_BB_size[0] / object_BB_size[1]
    object_y_z_ratio = object_BB_size[1] / object_BB_size[2]
    object_x_z_ratio = object_BB_size[0] / object_BB_size[2]
    
    shape_x_y_ratio = shape_BB_size[0] / shape_BB_size[1]
    shape_y_z_ratio = shape_BB_size[1] / shape_BB_size[2]
    shape_x_z_ratio = shape_BB_size[0] / shape_BB_size[2]
    
    if  abs(object_x_y_ratio - shape_x_y_ratio) < threshold and \
        abs(object_y_z_ratio - shape_y_z_ratio) < threshold and \
        abs(object_x_z_ratio - shape_x_z_ratio) < threshold:   
        return True

    return False

In [7]:
def search_shapenet(mesh_bb_mapping, object_name, object_size, threshold):
    res_paths = None
    mesh_paths = readData(shapenet_data_path + object_name + '/')
    res_paths = []
    for mesh_path in mesh_paths:
        bb = mesh_bb_mapping[mesh_path]
        if compare_BBs(bb.get_extent(), object_size, threshold):
            #within threshold
            res_paths.append(mesh_path)
            
    return res_paths

In [8]:
def calculate_match_error(object_BB_size, shape_BB_size):
    object_x_y_ratio = object_BB_size[0] / object_BB_size[1]
    object_y_z_ratio = object_BB_size[1] / object_BB_size[2]
    object_x_z_ratio = object_BB_size[0] / object_BB_size[2]
    
    shape_x_y_ratio = shape_BB_size[0] / shape_BB_size[1]
    shape_y_z_ratio = shape_BB_size[1] / shape_BB_size[2]
    shape_x_z_ratio = shape_BB_size[0] / shape_BB_size[2]
    error = (object_x_y_ratio - shape_x_y_ratio)**2 + \
            (object_y_z_ratio - shape_y_z_ratio)**2 + \
            (object_x_z_ratio - shape_x_z_ratio)**2
    return math.sqrt(error)

In [9]:
def get_sorted_args_for_shaped_matches(mesh_bb_mapping, res_paths, object_BB_size):
    bb_size_list = []
    for res_path in res_paths:
        shape_BB = mesh_bb_mapping[res_path]
        cur_bb_error = calculate_match_error(object_BB_size, shape_BB.get_extent())
        bb_size_list.append(cur_bb_error)
    return np.argsort(np.asarray(bb_size_list)), bb_size_list

In [10]:
def get_mesh_bb_mapping(shapenet_data_path, included_classes):
    mapping = {}
    for class_name in included_classes:
        obj_paths = readData(osp.join(shapenet_data_path, class_name))
        for obj_path in obj_paths:
            mapping[obj_path] = getBBFromMeshPath(obj_path)
    return mapping

In [51]:
user = 'baykara'
if user == 'gogebakan':
    base_path = '/home/gogebakan/workspace/Pose2Room/'
    train_path = base_path + 'datasets/virtualhome_22_classes/splits/script_level/train.json'
    validation_path = base_path + 'datasets/virtualhome_22_classes/splits/script_level/val.json'
    # shapenet_data_path = '/home/gogebakan/workspace/pointnet_pytorch/data/myshapenet/raw_obj/'
    shapenet_data_path = '/home/gogebakan/workspace/pointnet_pytorch/data/myshapenet/small_dataset/'
else:
    base_path = '/home/baykara/adl4cv/Pose2Room/'
    train_path = base_path + 'datasets/virtualhome_22_classes/splits/script_level/train.json'
    validation_path = base_path + 'datasets/virtualhome_22_classes/splits/script_level/val.json'
    shapenet_data_path = '/home/baykara/adl4cv/pointnet_pytorch/data/adl_shapenet/watertight'
    
included_classes = ['bench', 'cabinet', 'faucet', 'stove', 'bookshelf', 'computer', 'desk', 'chair', 'monitor', 'sofa', 'lamp', 'nightstand', 'bed', 'dishwasher', 'fridge', 'microwave', 'toilet']

class_thresholds = {
    'bench': 0.05,
    'cabinet': 0.05,
    'faucet': 0.05,
    'stove': 0.05,
    'bookshelf': 0.05,
    'computer': 0.05,
    'desk': 0.05,
    'chair': 0.35, # min 9 max 3203 
    'monitor': 0.05,
    'sofa': 0.5, # min 22
    'lamp': 1.6, # min 9 
    'nightstand': 0.05,
    'bed': 1.5, # min 14
    'dishwasher': 0.05,
    'fridge': 0.05,
    'microwave': 0.05,
    'toilet': 0.05
    }

In [27]:
mesh_bb_mapping = get_mesh_bb_mapping(shapenet_data_path, included_classes)

Opening JSON file

In [69]:
for json_path in [train_path, validation_path]:
    p2r_classes = []
    with open(json_path) as f:
        # returns JSON object as a dictionary
        path_list = json.load(f)

    logging.info(f'Path list size: {len(path_list)}')
    for current_hdf in tqdm(path_list):
        logging.debug(f'Current scene: {current_hdf}')
        sample_data = h5py.File(osp.join(base_path, current_hdf), "a")
        object_nodes = get_object_nodes(sample_data)
        shape_codes = []
        # process object nodes
        for object_node in object_nodes:
            cur_size = object_node['size']
            class_name = object_node['class_name'][0].decode("utf-8")
            if class_name not in p2r_classes:
                p2r_classes.append(class_name)
    print(p2r_classes)  
        #     # if class_name in included_classes:
        #     if class_name == 'table':
        #         res_paths = search_shapenet(mesh_bb_mapping, class_name, cur_size, 0.5)
        #         # res_paths = search_shapenet(mesh_bb_mapping, class_name, cur_size, class_thresholds[class_name])
        #         print(f'{class_name} : {len(res_paths)}')
                
        #         with open('res_paths.json', 'w') as f:
        #             json.dump(res_paths, f)
        #         #sort
        #         sorted_args, bb_size_list = get_sorted_args_for_shaped_matches(mesh_bb_mapping, res_paths, cur_size)
        #         #mesh penetration loss
        #         #TO-DO
                
        #         best_match = res_paths[sorted_args[0]]
        #         best_match = os.path.splitext(best_match)[0] + '.npy'
        #         shape_code = np.load(best_match)
        #         shape_codes.append(shape_code)
        #         logging.debug(f'Object: {class_name}\tBest match: {best_match}')
        #     else:
        #         shape_codes.append(np.zeros((1, 1024)))

        # # shape codes for each scene have to be in size (10, 1024)
        # for i in range(len(shape_codes), 10):
        #     shape_codes.append(np.zeros((1, 1024)))
        # if 'shape_codes' in sample_data.keys():
        #     del sample_data['shape_codes']
        # sample_data.create_dataset('shape_codes', data=shape_codes) 
        # sample_data.close()
        

INFO:root:Path list size: 2781
100%|██████████| 2781/2781 [01:15<00:00, 37.04it/s]


In [70]:
print(p2r_classes)

['sofa', 'cabinet', 'faucet', 'fridge', 'stove', 'microwave', 'nightstand', 'bench', 'chair', 'bookshelf', 'dishwasher', 'lamp', 'computer', 'monitor', 'desk', 'bed', 'toilet']
